## Setup Environment <br>
<br>
• Torch - 1.6.0<br>
• Torchvision – 0.7.0<br>
• CUDA – 10.1<br>
• CuDNN – v7.6.5.32 for CUDA 10.1<br>
• Sklearn – 0.22.2.post1<br>
• MLFlow – 1.10.0<br>
• numpy – 1.18.5<br>

In [ ]:
import torch
import torch.nn as nn
from torch.utils import data
import torchvision
import torchvision.datasets
import sklearn
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np 
import mlflow
import mlflow.pytorch